In [1]:
# Imports
import geemap
import numpy as np
import pandas as pd
import ee

In [2]:
js_snippet = """


// var scollection=ee.FeatureCollection([sunderban])
// Export.table.toDrive({
//   collection:scollection,
//   description:'Shapefile',
//   fileFormat:'SHP'
// })
var thane = ee.FeatureCollection([sunderban])
var s2 = ee.ImageCollection("COPERNICUS/S2")

var rgbVis = {
  min: 0.0,
  max: 3000,
  bands: ['B4', 'B3', 'B2'],
};

// Write a function for Cloud masking
function maskS2clouds(image) {
  var qa = image.select('QA60')
  var cloudBitMask = 1 << 10;
  var cirrusBitMask = 1 << 11;
  var mask = qa.bitwiseAnd(cloudBitMask).eq(0).and(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask)//.divide(10000)
      .select("B.*")
      .copyProperties(image, ["system:time_start"])
} 

var filtered = s2
  .filter(ee.Filter.date('2015-01-01', '2015-12-31'))
  .filter(ee.Filter.bounds(thane))
  .map(maskS2clouds)

var timelapseimage = s2
  .filter(ee.Filter.bounds(thane))
  .map(maskS2clouds)

var filtered2 = s2
  .filter(ee.Filter.date('2020-01-01', '2020-12-31'))
  .filter(ee.Filter.bounds(thane))
  .map(maskS2clouds)
  
var image2019 = filtered.median().clip(thane)
var image2020 = filtered2.median().clip(thane)

// Display the input composite.

Map.addLayer(image2019, rgbVis, '2019');
Map.addLayer(image2020, rgbVis, '2020');



var ndvi2019 = image2019.normalizedDifference(['B8', 'B4']).rename(['ndvi']);
var ndvi2020 = image2020.normalizedDifference(['B8', 'B4']).rename(['ndvi']);
var ndviVis = {min:0, max:1, palette: ['white', 'green']}

 Map.addLayer(ndvi2019, ndviVis, 'ndvi')
 Map.addLayer(ndvi2020, ndviVis, 'ndvi')
 
// var mean_ndvi = ndvi.reduceRegion({
//   reducer: ee.Reducer.mean(),
//   geometry: thane,
//   maxPixels:111768980,
//   scale: 30
// });

// var sd_ndvi = ndvi.reduceRegion({
//   reducer: ee.Reducer.stdDev(),
//   geometry: thane,
//   maxPixels:111768980,
//   scale: 30
// });

// print(ndvi) 
// print('Mean:',mean_ndvi);
// print('SD',sd_ndvi);

// 3. Classify NDVI into 5 classes
var ndvi2019classify = ee.Image(1)
          .where(ndvi2019.gt(0.0).and(ndvi2019.lte(0.2)), 2)
          .where(ndvi2019.gt(0.2).and(ndvi2019.lte(0.4)), 3)
          .where(ndvi2019.gt(0.4).and(ndvi2019.lte(0.6)), 4)
          .where(ndvi2019.gt(0.6), 5).clip(thane)
          
var ndvi2020classify = ee.Image(1)
          .where(ndvi2020.gt(0.0).and(ndvi2020.lte(0.2)), 2)
          .where(ndvi2020.gt(0.2).and(ndvi2020.lte(0.4)), 3)
          .where(ndvi2020.gt(0.4).and(ndvi2020.lte(0.6)), 4)
          .where(ndvi2020.gt(0.6), 5).clip(thane)

// Add map layers
Map.addLayer(ndvi2019classify, {min: 1, max: 5, palette: ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Classified NDVI 2019',true);
Map.addLayer(ndvi2020classify, {min: 1, max: 5, palette: ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Classified NDVI 2020',true);

var mapTitle = ui.Panel({
  style: {
    position: 'top-center',
    padding: '8px 15px'
  }
});
var mapTitle2 = ui.Label({
  value: 'Map of NDVI',
  style: {
    fontWeight: 'bold',
    fontSize: '20px',
    margin: '0 0 3px 0',
    padding: '0'
    }
});
mapTitle.add(mapTitle2);
Map.add(mapTitle);
// Add map legend
var legend = ui.Panel({
  style: {
    position: 'bottom-right',
    padding: '8px 15px'
  }
});
var legend2 = ui.Label({
  value: 'Legend (NDVI)',
  style: {
    fontWeight: 'bold',
    fontSize: '18px',
    margin: '0 0 4px 0',
    padding: '0'
    }
});
legend.add(legend2);
// Creates the content of the legend
var content = function(color, label) {
      // Create the color boxes
      var box = ui.Label({
        style: {
          backgroundColor: '#' + color,
          // Set box height and width
          padding: '9px',
          margin: '0 0 4px 0'
        }
      });
      // Create the labels
      var labels = ui.Label({
        value: label,
        style: {margin: '0 0 4px 6px'}
      });
      return ui.Panel({
        widgets: [box, labels],
        layout: ui.Panel.Layout.Flow('horizontal')
      });
};
//  Set legend colors
var classcolor = ['654321','FFA500','FFFF00', '00FF00', '008000'];
// Set legend labels
var labelName = ['<=0','0 - 0.2','0.2 - 0.4','0.4 - 0.6', '>0.6'];
// Combine legend colou and labels
for (var i = 0; i < 5; i++) {
  legend.add(content(classcolor[i], labelName[i]));
  }  
// Add legend
Map.add(legend);

//spectral changes 
var magnitude = function(image) {
  return image.pow(2).reduce(ee.Reducer.sum()).sqrt();
};

var distance = ndvi2020.subtract(ndvi2019)

// Calculate Angle
// https://byjus.com/angle-between-two-vectors-formula/
// var dot = ndvi2019.multiply(ndvi2020).reduce(ee.Reducer.sum());

// var angle = dot.divide(magnitude(ndvi2020))
//               .divide(magnitude(ndvi2019))
//               .acos()

Map.addLayer(distance, {min: 0, max: 0.1, palette: ['white', 'green']}, 'spectral distance');
// Map.addLayer(angle, {min: 0, max: 1, palette: ['white', 'purple']}, 'angle');





"""

In [5]:
geemap.js_snippet_to_py(
    js_snippet, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True
)

In [14]:
import ee
import geemap
import ipywidgets as widgets

ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [17]:


admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2");
thane = admin2.filter(ee.Filter.eq('ADM2_NAME', 'Thane'))
Map = geemap.Map()

# scollection=ee.FeatureCollection([sunderban])
# Export.table.toDrive({
#   collection:scollection,
#   description:'Shapefile',
#   fileFormat:'SHP'
# })

s2 = ee.ImageCollection("COPERNICUS/S2")

rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

# Write a function for Cloud masking
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).select("B.*") \
      .copyProperties(image, ["system:time_start"])

filtered = s2 \
  .filter(ee.Filter.date('2019-01-01', '2019-12-31')) \
  .filter(ee.Filter.bounds(thane)) \
  .map(maskS2clouds)

timelapseimage = s2 \
  .filter(ee.Filter.bounds(thane)) \
  .map(maskS2clouds)

filtered2 = s2 \
  .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
  .filter(ee.Filter.bounds(thane)) \
  .map(maskS2clouds)

image2019 = filtered.median().clip(thane)
image2020 = filtered2.median().clip(thane)

# Display the input composite.

Map.addLayer(image2019, rgbVis, '2019')
Map.addLayer(image2020, rgbVis, '2020')

ndvi2019 = image2019.normalizedDifference(['B8', 'B4']).rename(['ndvi'])
ndvi2020 = image2020.normalizedDifference(['B8', 'B4']).rename(['ndvi'])
ndviVis = {'min':0, 'max':1, 'palette': ['white', 'green']}

Map.addLayer(ndvi2019, ndviVis, 'ndvi')
Map.addLayer(ndvi2020, ndviVis, 'ndvi')

# mean_ndvi = ndvi.reduceRegion({
#   reducer: ee.Reducer.mean(),
#   geometry: thane,
#   maxPixels:111768980,
#   scale: 30
# })

# sd_ndvi = ndvi.reduceRegion({
#   reducer: ee.Reducer.stdDev(),
#   geometry: thane,
#   maxPixels:111768980,
#   scale: 30
# })

# print(ndvi)
# print('Mean:',mean_ndvi)
# print('SD',sd_ndvi)

# 3. Classify NDVI into 5 classes
ndvi2019classify = ee.Image(1) \
          .where(ndvi2019.gt(0.0).And(ndvi2019.lte(0.2)), 2) \
          .where(ndvi2019.gt(0.2).And(ndvi2019.lte(0.4)), 3) \
          .where(ndvi2019.gt(0.4).And(ndvi2019.lte(0.6)), 4) \
          .where(ndvi2019.gt(0.6), 5).clip(thane)

ndvi2020classify = ee.Image(1) \
          .where(ndvi2020.gt(0.0).And(ndvi2020.lte(0.2)), 2) \
          .where(ndvi2020.gt(0.2).And(ndvi2020.lte(0.4)), 3) \
          .where(ndvi2020.gt(0.4).And(ndvi2020.lte(0.6)), 4) \
          .where(ndvi2020.gt(0.6), 5).clip(thane)

# Add map layers
Map.addLayer(ndvi2019classify, {'min': 1, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Classified NDVI 2019',True)
Map.addLayer(ndvi2020classify, {'min': 1, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Classified NDVI 2020',True)

# mapTitle = ui.Panel({
#   'style': {
#     'position': 'top-center',
#     'padding': '8px 15px'
#   }
# })
# mapTitle2 = ui.Label({
#   'value': 'Map of NDVI',
#   'style': {
#     'fontWeight': 'bold',
#     'fontSize': '20px',
#     'margin': '0 0 3px 0',
#     'padding': '0'
#     }
# })
# mapTitle.add(mapTitle2)
# Map.add(mapTitle)
# # Add map legend
# legend = ui.Panel({
#   'style': {
#     'position': 'bottom-right',
#     'padding': '8px 15px'
#   }
# })
# legend2 = ui.Label({
#   'value': 'Legend (NDVI)',
#   'style': {
#     'fontWeight': 'bold',
#     'fontSize': '18px',
#     'margin': '0 0 4px 0',
#     'padding': '0'
#     }
# })
# legend.add(legend2)
# # Creates the content of the legend
# def content(color, label):
#       # Create the color boxes
#       box = ui.Label({
#         'style': {
#           'backgroundColor': '#' + color,
#           # Set box height and width
#           'padding': '9px',
#           'margin': '0 0 4px 0'
#         }
#       })
#       # Create the labels
#       labels = ui.Label({
#         'value': label,
#         'style': {'margin': '0 0 4px 6px'}
#       })

#       return ui.Panel({
#         'widgets': [box, labels],
#         'layout': ui.Panel.Layout.Flow('horizontal')
#       })

# #  Set legend colors
# classcolor = ['654321','FFA500','FFFF00', '00FF00', '008000']
# # Set legend labels
# labelName = ['<=0','0 - 0.2','0.2 - 0.4','0.4 - 0.6', '>0.6']
# # Combine legend colou and labels
# for i in range(0, 5, 1):
#   legend.add(content(classcolor[i], labelName[i]))

# # Add legend
# Map.add(legend)

#spectral changes
def magnitude(image):
  return image.pow(2).reduce(ee.Reducer.sum()).sqrt()

distance = ndvi2020.subtract(ndvi2019)

# Calculate Angle
# https:#byjus.com/angle-between-two-vectors-formula/
# dot = ndvi2019.multiply(ndvi2020).reduce(ee.Reducer.sum())

# angle = dot.divide(magnitude(ndvi2020))
#               .divide(magnitude(ndvi2019))
#               .acos()

Map.addLayer(distance, {'min': 0, 'max': 0.1, 'palette': ['white', 'green']}, 'spectral distance')
# Map.addLayer(angle, {min: 0, max: 1, palette: ['white', 'purple']}, 'angle')

In [18]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [28]:
label="Timelapse for NDVI"
roi= ee.Geometry.Polygon(
        [[[73.04124316585163, 18.97918033662206],
          [73.94761523616413, 19.33721584963624],
          [73.31590136897663, 20.308722406734972],
          [72.53037890803913, 20.154094798779248],
          [72.64573535335163, 19.311297428069476]]]);
Map.add_landsat_ts_gif(label= label,roi=roi, bands=['Blue', 'NIR', 'Red'], nd_bands=['NIR', 'Red'], nd_palette=['blue', 'white', 'green'], nd_threshold=0, start_year=1985, end_year=2020,start_date="01-01", end_date="12-31",add_progress_bar= True, frames_per_second=3)


Generating URL...
Please wait ...
The GIF image has been saved to: /var/folders/83/rcfkd72n2k3cmzcj288kt_h80000gn/T/d292c66c-6df0-4e85-b624-5e1be4463ff6.gif
Generating URL...
Please wait ...
The GIF image has been saved to: /var/folders/83/rcfkd72n2k3cmzcj288kt_h80000gn/T/d292c66c-6df0-4e85-b624-5e1be4463ff6_nd.gif
Adding GIF to the map ...
The timelapse has been added to the map.
